In [8]:
import sklearn
import pandas as pd
import numpy as np
!pip install catboost
import catboost
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

In [5]:
data_main=pd.read_csv('data_with_concur.csv')
data_main.drop(['Unnamed: 0'], axis=1, inplace=True)

In [3]:
cian_data=pd.read_csv('cian_data.csv')

In [10]:
cian=cian_data.groupby([pd.cut(cian_data['latitude'], np.arange(55.655155, 55.753211975097656+0.0028016226632254465, 0.0028016226632254465)),pd.cut(cian_data['longitude'], np.arange(37.49541473388672, 37.75920104980469+0.007536751883370536, 0.007536751883370536))])['price'].mean().reset_index()

In [12]:
cian.price=cian.price.fillna(10000000)

In [13]:
data_main['price']=np.zeros(data_main.shape[0])

In [24]:
data_main['price']=cian['price']

In [25]:
data_main.to_csv('data_main_train.csv')

In [ ]:
spheres=data_main.columns[9:-1]

In [133]:
data_bez_con=data_main.drop(spheres, axis=1)

In [211]:
data_bez_con.to_csv('model_dataset.csv')

In [136]:
train=[]
for i in range(data_main.shape[0]):
  for j in spheres:
    if data_main.iloc[i][j]>0:
      train.append([*data_bez_con.iloc[i].to_list(),j,1])
    else:
      if random.random()>0.8:
        train.append([*data_bez_con.iloc[i].to_list(),j,0])

In [137]:
data_train=pd.DataFrame(train)

In [138]:
data_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,"(55.655, 55.658]","(37.495, 37.503]",0.0,0.0,0.0,0.0,0.0,Юго-западная,0.016882,93104.0,cafe,1
1,"(55.655, 55.658]","(37.495, 37.503]",0.0,0.0,0.0,0.0,0.0,Юго-западная,0.016882,93104.0,clinic,1
2,"(55.655, 55.658]","(37.495, 37.503]",0.0,0.0,0.0,0.0,0.0,Юго-западная,0.016882,93104.0,dentist,1
3,"(55.655, 55.658]","(37.503, 37.51]",0.0,0.0,0.0,0.0,0.0,Проспект Вернадского,0.019732,10000000.0,drink,0
4,"(55.655, 55.658]","(37.51, 37.518]",0.0,0.0,0.0,0.0,0.0,Беляево,0.018684,10000000.0,drink,1


In [139]:
X=data_train.drop([11],axis=1)

In [140]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10
0,"(55.655, 55.658]","(37.495, 37.503]",0.0,0.0,0.0,0.0,0.0,Юго-западная,0.016882,93104.0,cafe
1,"(55.655, 55.658]","(37.495, 37.503]",0.0,0.0,0.0,0.0,0.0,Юго-западная,0.016882,93104.0,clinic
2,"(55.655, 55.658]","(37.495, 37.503]",0.0,0.0,0.0,0.0,0.0,Юго-западная,0.016882,93104.0,dentist
3,"(55.655, 55.658]","(37.503, 37.51]",0.0,0.0,0.0,0.0,0.0,Проспект Вернадского,0.019732,10000000.0,drink
4,"(55.655, 55.658]","(37.51, 37.518]",0.0,0.0,0.0,0.0,0.0,Беляево,0.018684,10000000.0,drink


In [152]:
y=data_train[11]

In [157]:
spheres

Index(['bakery', 'butcher', 'cafe', 'chemist', 'clinic', 'dentist', 'drink',
       'fixer', 'florist', 'hairdresser', 'pharmacy', 'photo', 'shop', 'spa',
       'tobacco', 'vets'],
      dtype='object')

In [163]:
cb=catboost.CatBoostClassifier(cat_features=[0,1,7,10])

In [164]:
X_train, X_test, y_train, y_test= train_test_split(X, y)

In [ ]:
cb.fit(X_train, y_train)

In [166]:
roc_auc_score(y_test, cb.predict_proba(X_test)[:,1])

0.9035452404978137

In [167]:
accuracy_score(y_test, cb.predict(X_test))

0.8287444933920705